In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import neweggutils as nu
import re

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_csv('final.csv')

In [4]:
df.drop(labels=['Unnamed: 0'], inplace=True, axis=1)

In [6]:
df_clean = df.copy()

# Drop duplicates and unnecessary columns

In [9]:
df_clean = df_clean[~df_clean.duplicated()]

In [12]:
df_clean.columns.values

array(['price', 'link', 'Brand', 'Series', 'Model', 'Part Number', 'Type',
       'Form Factor', 'Usage', 'Colors', 'Processor',
       'Processor Main Features', 'Cache Per Processor', 'Memory',
       'Storage', 'Optical Drive', 'Graphics', 'Power Supply',
       'Operating System', 'CPU Type', 'CPU Speed', 'L3 Cache Per CPU',
       'CPU Main Features', 'GPU/VGA Type', 'Video Memory',
       'Virtual Reality Ready', 'Memory Capacity', 'Memory Speed',
       'Memory Spec', 'Memory Slot (Total)', 'Maximum Memory Supported',
       'Optane Memory', 'HDD', 'HDD RPM', 'Optical Drive Type',
       'Screen Size', 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type',
       'Keyboard Type', 'Dimensions (H x W x D)', 'Weight',
       'Graphics Interface', 'SSD', 'Keyboard', 'Storage Spec',
       'Optical Drive Spec', 'Media Slots', 'Display Feature',
       'Front USB', 'Mouse', 'Special Features', 'Package Contents',
       'Software Included', 'Audio Chipset', 'LAN Chipset', 'Video Ports',
    

In [15]:
df_clean = df_clean.drop(labels=['Series', 'Model', 'Part Number', 'Model', 'Colors', 'Cache Per Processor',
                                 'Optical Drive', 'L3 Cache Per CPU', 'Virtual Reality Ready', 'Optane Memory',
                                 'LAN Speed', 'WLAN', 'Bluetooth', 'Mouse Type', 'Keyboard Type',
                                 'Dimensions (H x W x D)', 'Keyboard', 'Mouse', 'Special Features',
                                 'Package Contents', 'Software Included', 'Audio Chipset', 'LAN Chipset',
                                 'Speaker Configuration', 'Front Audio Ports', 'L2 Cache Per CPU', 
                                 'COM', 'Speaker Features', 'FIR', 'Card Reader', 'Installed Qty', 'Touchscreen',
                                 'LPT', 'External Bays', 'Internal Bays', 'Package Type', 'Optical Drive 2',
                                 'Optical Drive 2 Type', 'Front IEEE 1394'], axis=1)

In [17]:
df_clean = df_clean.drop(labels=['Power Supply', 'Memory Speed', 'Memory Spec', 'Memory Slot (Total)',
                                 'Maximum Memory Supported', 'HDD RPM', 'Weight', 'Media Slots', 'Display Feature',
                                 'HDD Interface', 'Ethernet', 'Chipset', 'Audio Features', 'Memory Slot (Available)',
                                 'Graphics Card', 'Screen Type', 'PCI Slots (Available/Total)', 'Monitor',
                                 'Audio', 'Speaker', 'SLI / Crossfire', 'Max Supported Qty'], axis=1)

In [26]:
df_clean.to_csv('final_pared.csv')

In [42]:
df_clean.reset_index(inplace=True, drop=True)

In [44]:
df_clean['proc_brand'] = df_clean['Processor'].str.extract(r'^(INTEL|AMD)', flags=re.IGNORECASE)

In [74]:
df_clean.drop(labels=['Type', 'Usage'], axis=1, inplace=True)

In [88]:
df_clean['Processor Main Features'].str.extract(r'(-Core|Core)', flags=re.IGNORECASE)

0    0.252858
dtype: float64

In [141]:
df_clean['proc_cores'] = df_clean['Processor'].apply(lambda x: nu.num_cores(str(x)))

In [158]:
pat = re.compile('([0-9]\.[0-9]?)')
df_clean['proc_speed'] = df_clean['Processor'].str.extract(pat)

In [63]:
t = df_clean['Processor'].apply(lambda x: str(x).split(' ')[-1])

In [71]:
df_clean['Processor'].str.extract(r'(CORE)', flags=re.IGNORECASE)

,0
0,Core
1,Core
2,Core
3,Core
4,NaN
5,Core
6,Core
7,Core
8,Core
9,Core
